<a href="https://colab.research.google.com/github/Vandan1009/python_1/blob/main/pspyk2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SparkSQL

Working with PySpark's built-in SQL compliant functionality to investigate flights-delay data at scale


*   Understand the limitations of SparkSQL
*   Experiment with createOrReplaceGlobalView
*   Work on same executions through pyspark methods



In [ ]:
#Download Data from source
Data Source Repo - https://kloudbitbucket.s3.amazonaws.com/krunal_ds/departuredelays.csv

In [1]:
#Import necessary spark components
!pip install pyspark



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=c97a19a220cc766bd690d2df7f0059c73ab6c09db6a1ae3f2f63d050929517b4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
#Create a SparkSession
spark=SparkSession.builder.appName("notebook2").getOrCreate()

In [7]:
#Import Data into Spark Native Dataframe
df=spark.read.option('Header','True').csv('/content/departuredelays (1).csv')

In [9]:
#Convert into an SQL complaint format
sd=df
sd.createOrReplaceTempView('delays')


In [10]:
#Display 100 rows of data with SQL query
spark.sql("select * from delays ").limit(100).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

In [11]:
#Get a list of all Origin and Destination airports
spark.sql("select origin,destination from delays ").show()

+------+-----------+
|origin|destination|
+------+-----------+
|   ABE|        ATL|
|   ABE|        DTW|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        ATL|
|   ABE|        DTW|
|   ABE|        ATL|
|   ABE|        DTW|
|   ABE|        ATL|
|   ABE|        DTW|
|   ABE|        ATL|
|   ABE|        ORD|
|   ABE|        DTW|
+------+-----------+
only showing top 20 rows



In [13]:
#Find the top 5 longest distance travel routes
spark.sql("select * from delays order by distance desc limit 5").show()


+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01021955|  116|     998|   LGA|        OMA|
|01072030|   45|     998|   LGA|        OMA|
|01031955|   65|     998|   LGA|        OMA|
|01181500|   -3|     998|   AUS|        DTW|
|01051949|    0|     998|   LGA|        OMA|
+--------+-----+--------+------+-----------+



In [20]:
#Find top 3 origin airport with maximum flight delay occurances
spark.sql("select origin from dalays group by origin order by sum(delay) desc limit 3 ").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `dalays` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 19;
'GlobalLimit 3
+- 'LocalLimit 3
   +- 'Sort ['sum('delay) DESC NULLS LAST], true
      +- 'Aggregate ['origin], ['origin]
         +- 'UnresolvedRelation [dalays], [], false


In [19]:
#Find top 3 destination airport with maximum flight delay occurances
spark.sql("select destination from delays group by destination order by sum(delay) desc limit 3 ").show()


+-----------+
|destination|
+-----------+
|        ATL|
|        ORD|
|        DEN|
+-----------+



In [24]:
 #Find the route with maximum delay occurances
spark.sql("select orgini,destination, max(delay) from delays group by orgini,destination order by max(delay)desc limit 1").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `orgini` cannot be resolved. Did you mean one of the following? [`origin`, `date`, `delay`, `distance`, `destination`].; line 1 pos 7;
'GlobalLimit 1
+- 'LocalLimit 1
   +- 'Sort ['max('delay) DESC NULLS LAST], true
      +- 'Aggregate ['orgini, destination#21], ['orgini, destination#21, max(delay#18) AS max(delay)#181]
         +- SubqueryAlias delays
            +- View (`delays`, [date#17,delay#18,distance#19,origin#20,destination#21])
               +- Relation [date#17,delay#18,distance#19,origin#20,destination#21] csv


In [22]:
#Find the top three routes with maximum time-delay
spark.sql("select * from delays order by delay desc limit 3").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01061635|   99|     516|   ATL|        DTW|
|01062008|   99|     586|   ATL|        TUL|
|01061518|   99|    1042|   ATL|        DEN|
+--------+-----+--------+------+-----------+



In [23]:
#Find the distance for top three max time-delay routes
spark.sql("select * from delays order by distance desc limit 3").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01111500|   99|     998|   AUS|        DTW|
|01251500|   -7|     998|   AUS|        DTW|
|01181500|   -3|     998|   AUS|        DTW|
+--------+-----+--------+------+-----------+

